# FlashRank Reranker로 문서 재순위화

## 개요
FlashRank는 검색 및 retrieval 파이프라인에 재순위를 추가하기 위한 초경량 및 초고속 Python 라이브러리입니다. 이 노트북에서는 FlashRank를 사용하여 문서 압축 및 retrieval을 수행하는 방법을 알아봅니다.

### 주요 특징
- **초경량**: 최소한의 메모리 사용
- **초고속**: 빠른 재순위 처리
- **SoTA cross-encoders 기반**: 최신 cross-encoder 모델 활용
- **쉬운 통합**: LangChain과 원활한 통합

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
# !pip install -qU flashrank
# !pip install -qU langchain langchain-community langchain-openai faiss-cpu

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정 (필요시)
# os.environ["OPENAI_API_KEY"] = "your-api-key"

## 2. 유틸리티 함수 정의

In [2]:
def pretty_print_docs(docs):
    """문서를 보기 좋게 출력하는 유틸리티 함수"""
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

## 3. 데이터 로드 및 전처리

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # 청크 크기
    chunk_overlap=100    # 청크 간 중복
)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 각 텍스트에 고유 ID 추가
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

print(f"총 {len(texts)}개의 텍스트 청크가 생성되었습니다.")

## 4. 벡터 스토어 및 기본 Retriever 생성

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(texts, embeddings)

# 기본 retriever 생성 (상위 10개 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

## 5. 기본 Retriever 테스트

In [5]:
# 질의문
query = "Word2Vec에 대해서 설명해줘."

# 기본 retriever로 문서 검색
docs = retriever.invoke(query)

print(f"\n질의: {query}")
print(f"\n검색된 문서 개수: {len(docs)}")
print(f"검색된 문서 ID: {[doc.metadata['id'] for doc in docs]}")
print("\n=== 검색된 문서 내용 ===\n")
pretty_print_docs(docs[:3])  # 상위 3개만 출력

## 6. FlashRank Reranker 적용

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

# FlashRank reranker 초기화
# ms-marco-MultiBERT-L-12: MS MARCO 데이터셋으로 학습된 다국어 BERT 모델
compressor = FlashrankRerank(
    model="ms-marco-MultiBERT-L-12",
    top_n=3  # 상위 3개 문서만 반환 (옵션)
)

# ContextualCompressionRetriever로 래핑
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

## 7. Reranker 적용 결과 비교

In [7]:
# FlashRank reranker로 압축된 문서 검색
compressed_docs = compression_retriever.invoke(query)

print(f"\n질의: {query}")
print(f"\n재순위화 후 문서 개수: {len(compressed_docs)}")
print(f"재순위화 후 문서 ID: {[doc.metadata['id'] for doc in compressed_docs]}")
print("\n=== 재순위화된 문서 내용 ===\n")
pretty_print_docs(compressed_docs)

## 8. Relevance Score 확인

FlashRank는 각 문서에 relevance score를 추가합니다.

In [8]:
# Relevance score 확인
print("\n=== Relevance Scores ===\n")
for i, doc in enumerate(compressed_docs):
    score = doc.metadata.get('relevance_score', 'N/A')
    doc_id = doc.metadata.get('id', 'N/A')
    print(f"Document {i+1} (ID: {doc_id}): Score = {score:.6f}")
    # 문서 내용의 첫 100자만 출력
    preview = doc.page_content[:100] + "..."
    print(f"  Preview: {preview}\n")

## 9. 다양한 쿼리로 테스트

In [9]:
# 여러 쿼리 테스트
test_queries = [
    "임베딩이란 무엇인가?",
    "딥러닝과 머신러닝의 차이점",
    "판다스 DataFrame 사용법"
]

for test_query in test_queries:
    print(f"\n{'='*80}")
    print(f"질의: {test_query}")
    print(f"{'='*80}")
    
    # 기본 retriever
    basic_docs = retriever.invoke(test_query)
    print(f"\n기본 Retriever - 상위 3개 문서 ID: {[doc.metadata['id'] for doc in basic_docs[:3]]}")
    
    # FlashRank reranker
    reranked_docs = compression_retriever.invoke(test_query)
    print(f"FlashRank Reranker - 상위 3개 문서 ID: {[doc.metadata['id'] for doc in reranked_docs[:3]]}")
    
    # Relevance scores
    scores = [doc.metadata.get('relevance_score', 0) for doc in reranked_docs[:3]]
    print(f"Relevance Scores: {[f'{score:.4f}' for score in scores]}")

## 10. RAG 파이프라인에 통합

In [10]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini"
)

# 기본 retriever를 사용한 QA 체인
basic_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# FlashRank reranker를 사용한 QA 체인
reranked_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True
)

In [11]:
# 질문
question = "Word2Vec의 정의와 예시를 설명해주세요."

print(f"질문: {question}\n")
print("="*80)

# 기본 retriever 결과
print("\n### 기본 Retriever 사용 ###")
basic_result = basic_qa.invoke({"query": question})
print(f"답변: {basic_result['result']}")
print(f"\n사용된 문서 수: {len(basic_result['source_documents'])}")

print("\n" + "="*80)

# FlashRank reranker 결과
print("\n### FlashRank Reranker 사용 ###")
reranked_result = reranked_qa.invoke({"query": question})
print(f"답변: {reranked_result['result']}")
print(f"\n사용된 문서 수: {len(reranked_result['source_documents'])}")

## 11. 성능 비교

In [12]:
import time

def measure_retrieval_time(retriever, query, n_runs=5):
    """Retriever의 평균 실행 시간을 측정"""
    times = []
    for _ in range(n_runs):
        start = time.time()
        _ = retriever.invoke(query)
        end = time.time()
        times.append(end - start)
    return sum(times) / len(times)

# 성능 측정
query = "임베딩과 벡터화에 대해 설명해줘"

basic_time = measure_retrieval_time(retriever, query)
reranked_time = measure_retrieval_time(compression_retriever, query)

print("\n=== 성능 비교 ===")
print(f"기본 Retriever 평균 시간: {basic_time:.4f}초")
print(f"FlashRank Reranker 평균 시간: {reranked_time:.4f}초")
print(f"추가 소요 시간: {reranked_time - basic_time:.4f}초")
print(f"\n* FlashRank는 초경량 모델이므로 추가 시간이 매우 적습니다.")

## 12. FlashRank 모델 옵션

In [13]:
# 다양한 FlashRank 모델 테스트
models = [
    "ms-marco-TinyBERT-L-2-v2",      # 가장 작은 모델
    "ms-marco-MiniLM-L-12-v2",       # 중간 크기 모델
    "ms-marco-MultiBERT-L-12",       # 다국어 지원 모델
]

print("\n=== FlashRank 모델 비교 ===")
print(f"질의: {query}\n")

for model_name in models:
    try:
        # 모델별 compressor 생성
        model_compressor = FlashrankRerank(
            model=model_name,
            top_n=3
        )
        
        # Compression retriever 생성
        model_retriever = ContextualCompressionRetriever(
            base_compressor=model_compressor,
            base_retriever=retriever
        )
        
        # 시간 측정
        start = time.time()
        docs = model_retriever.invoke(query)
        end = time.time()
        
        print(f"\n모델: {model_name}")
        print(f"  - 실행 시간: {end - start:.4f}초")
        print(f"  - 상위 문서 ID: {[doc.metadata['id'] for doc in docs[:3]]}")
        if docs and 'relevance_score' in docs[0].metadata:
            print(f"  - 최고 점수: {docs[0].metadata['relevance_score']:.6f}")
    except Exception as e:
        print(f"\n모델 {model_name} 오류: {e}")

## 요약

### FlashRank Reranker의 장점

1. **초경량 & 초고속**: 매우 작은 모델 크기와 빠른 처리 속도
2. **높은 정확도**: Cross-encoder 기반으로 우수한 재순위화 성능
3. **쉬운 통합**: LangChain의 ContextualCompressionRetriever와 원활한 통합
4. **다국어 지원**: MultiBERT 모델을 통한 다국어 문서 처리

### 사용 시나리오

- **RAG 파이프라인 개선**: 검색된 문서의 품질 향상
- **리소스 제한 환경**: 서버리스, 엣지 디바이스 등에서 사용
- **실시간 애플리케이션**: 빠른 응답이 필요한 서비스
- **대규모 문서 처리**: 많은 문서 중 가장 관련성 높은 것만 선별

### 주의사항

- FlashRank는 재순위화만 수행하므로 초기 retrieval 품질이 중요
- top_n 파라미터로 반환할 문서 수 조절 가능
- 모델 선택 시 속도와 정확도 간 트레이드오프 고려